# Using EcoFOCIpy to process raw field data

## DY1805 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [9]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [10]:
sample_data_dir = '/Users/bell/ecoraid/2018/CTDcasts/dy1805/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [11]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1805' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1805.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd001test.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd012.btl
Processi

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd016.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd026.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd029.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd037.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd038.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd039.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd040.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd041.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd042.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd043.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd044.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd045.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd046.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd047.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1805/rawconverted/ctd048.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [12]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd002.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,turbwetntu0,prdm,scan,datetime
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.00000,7.000000,7.000000,7.000000,7.000000,7
mean,7.541486,99.574143,328.448571,7.520443,99.296571,327.532714,31.934100,31.936414,25.435314,25.437229,1.164086,3.03490,3.034400,1.061257,31.316571,8856.142857,2018-05-01 04:00:46.142857216
min,7.510700,98.978000,327.105000,7.487400,98.678000,326.092000,31.933600,31.936000,25.426300,25.428500,0.885500,2.95840,2.958500,0.585300,2.031000,4575.000000,2018-05-01 03:57:48
25%,7.515150,99.064000,327.299500,7.495250,98.790500,326.433000,31.933850,31.936150,25.427900,25.430100,1.067000,2.96580,2.966050,0.595200,14.970000,6721.000000,2018-05-01 03:59:17
50%,7.519600,99.127000,327.495000,7.501600,98.907000,326.711000,31.934000,31.936300,25.440800,25.442500,1.074600,2.97760,2.977600,0.779800,30.712000,8789.000000,2018-05-01 04:00:43
75%,7.573900,100.215000,329.862500,7.553650,99.954000,328.982500,31.934300,31.936700,25.441300,25.443000,1.272800,3.11885,3.118300,1.234950,46.410500,10501.500000,2018-05-01 04:01:55
max,7.582000,100.356000,330.216000,7.556300,100.002000,329.095000,31.934800,31.936900,25.441700,25.443400,1.508900,3.13900,3.136000,2.403400,63.712000,14184.000000,2018-05-01 04:04:28
std,0.033293,0.651548,1.452254,0.032309,0.635460,1.409614,0.000416,0.000353,0.007498,0.007227,0.227784,0.08580,0.085008,0.676962,22.353069,3235.535166,NaN


## Create BTL report file

In [13]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [14]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1805',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'R/V Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2018, 4, 29),
 'EndDate': datetime.date(2018, 5, 10),
 'Project': 'EcoFOCI',
 'ChiefScientist': 'Peter Proctor',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Survey',
 'CruiseYear': 2018,
 'ctdlogs_pdf_name': 'DY1805_CastLogs.pdf'}

In [15]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 22869,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY1805',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY1805',
 'Project': 'FOCI Spring Mooring Survey',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 56,
 'LatitudeMin': 52.28,
 'LongitudeDeg': 164,
 'LongitudeMin': 2.92,
 'GMTDay': 30,
 'GMTMonth': 'Apr',
 'GMTYear': 2018,
 'GMTTime': 69660,
 'DryBulb': 3.7,
 'RelativeHumidity': 98,
 'WetBulb': -99.9,
 'Pressure': 1013,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 230,
 'WindSpd': 21.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 65,
 'StationNameID': 'M2C',
 'MaxDepth': 72,
 'InstrumentSerialNos': 'Press SN =291, Pri Temp SN = 4379, Sec Temp SN =2376, Pri Cond SN = 04-2985, Sec Cond Sn =04-3127, PAR Sn =70547, Fluor Sn = FLNTUS-2057, pri O2 Sn = 1961, sec O2 Sn = 0904, Turbid SN = FLNTUS-2057',
 'Notes': 'Niskin 2 did not close - bottom cap\r\nNiskin 10 did not close\r\nRBRbrevio in Nis

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [16]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [17]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,sbeox1ml/l,sbeox1ps,sbox1mm/kg,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,pressure,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
6.0,7.3231,98.163,318.922,7.3037,97.903,318.078,32.0658,32.0681,25.4894,25.4912,0.8379,3.6111,3.6116,0.6895,29.727,24809.0,2018-04-30 19:29:05,ctd001


In [18]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'sbeox1ml/l',
       'sbeox1ps', 'sbox1mm/kg', 'salinity_ch1', 'salinity_ch2', 'sigma_t_ch1',
       'sigma_t_ch2', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'turbidity', 'pressure', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [19]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [20]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [21]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 80B 7.62 7.021 7.312 ... 7.327 7.325
    oxy_percentsat_ch1  (bottle) float64 80B 102.1 94.1 98.01 ... 98.24 98.23
    oxy_conc_ch1        (bottle) float64 80B 331.9 305.7 318.4 ... 319.1 319.0
    sbeox1ml/l          (bottle) float64 80B 7.279 7.287 7.302 ... 7.321 7.313
    sbeox1ps            (bottle) float64 80B 97.57 97.67 97.88 ... 98.15 98.06
    sbox1mm/kg          (bottle) float64 80B 317.0 317.4 318.0 ... 318.8 318.5
    ...                  ...
    temperature_ch2     (bottle) float64 80B 3.61 3.61 3.61 ... 3.632 3.62 3.627
    turbidity           (bottle) float64 80B 1.585 1.514 ... 0.6597 0.7356
    pressure            (bottle) float64 80B 65.42 65.16 51.24 ... 9.09 3.948
    scan                (bottle) float64 80B 1.269e+04 1.286e+04 ... 3.597e+04
    datetime            (bottle) datetime64[ns] 80B 2018-04-30T19:20:40 ... 2...
    cast                (bottle) object 80B 'ctd001' 'ctd001' ... 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [22]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [23]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [24]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 80B 7.62 ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80B 102.1...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 80B 331.9...
    salinity_ch1        (latitude, longitude, time, bottle) float64 80B 32.07...
    salinity_ch2        (latitude, longitude, time, bottle) float64 80B 32.07...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 80B 25.49...
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 80B 25.49...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 80B 0.823...
    temperature_ch1     (latitude, longitude, time, bottle) float64 80B 3.609...
    temperature_ch2     (latitude, longitude, time, bottle) float64 80B 3.61 ...
    turbidity           (latitude, longitude, time, bottle) float64 80B 1.585...
    pressure            (latitude, longitude, time, bottle) float64 80B 65.42...

In [25]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 80B 7.62 ...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80B 102.1...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 80B 331.9...
    salinity_ch1        (latitude, longitude, time, bottle) float64 80B 32.07...
    salinity_ch2        (latitude, longitude, time, bottle) float64 80B 32.07...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 80B 25.49...
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 80B 25.49...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 80B 0.823...
    temperature_ch1     (latitude, longitude, time, bottle) float64 80B 3.609...
    temperature_ch2     (latitude, longitude, time, bottle) float64 80B 3.61 ...
    turbidity           (latitude, longitude, time, bottle) float64 80B 1.585...
    pressure            (latitude, longitude, time, bottle) float64 80B 65.42...
Attributes: (12/44)
    CruiseID:                      DY1805
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        R/V Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-12-03T08:18:15Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [26]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

Skipping ctd001test.btl


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history